In [213]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np
import time

In [214]:
df = pd.read_csv("weather.csv")

In [215]:
print(df.sensor.unique())
df.head()

['temperature' 'irradiance' 'wind speed']


,event_start,belief_horizon_in_sec,event_value,sensor,unit
0,2020-11-01 00:00:00+00,-637,11.36,temperature,°C
1,2020-11-01 00:00:00+00,35362,11.06,temperature,°C
2,2020-11-01 00:00:00+00,38960,11.41,temperature,°C
3,2020-11-01 00:00:00+00,28162,11.25,temperature,°C
4,2020-11-01 00:00:00+00,2969,11.39,temperature,°C


In [216]:
df.describe()

,belief_horizon_in_sec,event_value
count,1.184860e+06,1.184860e+06
mean,7.922316e+04,5.341262e+01
std,5.235308e+04,1.330435e+02
min,-3.240000e+03,-8.970000e+00
25%,3.210000e+04,2.630000e+00
50%,7.536000e+04,7.170000e+00
75%,1.257000e+05,1.972000e+01
max,1.725470e+05,8.446301e+02


In [217]:
df.isnull().sum().sum()  # No null values

0

In [218]:
dfIr = df.loc[df["sensor"] == "irradiance"]
dfTem = df.loc[df["sensor"] == "temperature"]
dfWind = df.loc[df["sensor"] == "wind speed"]

In [219]:
type(df.event_start.values)
df.event_start

0          2020-11-01 00:00:00+00
1          2020-11-01 00:00:00+00
2          2020-11-01 00:00:00+00
3          2020-11-01 00:00:00+00
4          2020-11-01 00:00:00+00
                    ...          
1184855    2022-09-16 16:00:00+00
1184856    2022-09-16 16:00:00+00
1184857    2022-09-16 17:00:00+00
1184858    2022-09-16 17:00:00+00
1184859    2022-09-16 18:00:00+00
Name: event_start, Length: 1184860, dtype: object

In [220]:
df.event_start = pd.to_datetime(df.event_start).dt.tz_localize(None)

In [221]:
df.belief_horizon_in_sec

0            -637
1           35362
2           38960
3           28162
4            2969
            ...  
1184855    172540
1184856    165342
1184857    172539
1184858    168942
1184859    172542
Name: belief_horizon_in_sec, Length: 1184860, dtype: int64

In [222]:
# for i, k in enumerate(df.belief_horizon_in_sec):
#     # print(i, k)
df.belief_horizon_in_sec = pd.to_timedelta(df.belief_horizon_in_sec, unit='s')

In [223]:
df.belief_horizon_in_sec

0         -1 days +23:49:23
1           0 days 09:49:22
2           0 days 10:49:20
3           0 days 07:49:22
4           0 days 00:49:29
                 ...       
1184855     1 days 23:55:40
1184856     1 days 21:55:42
1184857     1 days 23:55:39
1184858     1 days 22:55:42
1184859     1 days 23:55:42
Name: belief_horizon_in_sec, Length: 1184860, dtype: timedelta64[ns]

In [224]:
df.belief_horizon_in_sec = df["event_start"] + df["belief_horizon_in_sec"]

In [225]:
df.belief_horizon_in_sec

0         2020-10-31 23:49:23
1         2020-11-01 09:49:22
2         2020-11-01 10:49:20
3         2020-11-01 07:49:22
4         2020-11-01 00:49:29
                  ...        
1184855   2022-09-18 15:55:40
1184856   2022-09-18 13:55:42
1184857   2022-09-18 16:55:39
1184858   2022-09-18 15:55:42
1184859   2022-09-18 17:55:42
Name: belief_horizon_in_sec, Length: 1184860, dtype: datetime64[ns]

In [226]:
# datetime_str = '09/19/22 13:55:26'
# datetime_object = datetime.strptime("2020/11/02 10:33:33", '%Y/%m/%d %H:%M:%S')
# print(datetime_object)
# datenow = datetime.strptime("20201102", '%y/%m/%d %H:%M:%S')
#  "%y/%m/%d %H:%M:%S"
datenow = "2020-11-02 10:00:00"
# datenow = datetime.strptime(datenow, '%Y-%m-%d %H:%M:%S')
# nowSlice = df.loc[(df.event_start <= datenow)]
# nowSlice

In [227]:
# bufdate = timedelta(hours=1)
# print(bufdate)
# datethen = "2020/11/03 10:00:00"
# datethen = datetime.strptime(datethen, '%Y/%m/%d %H:%M:%S')
#
# def datetime_range(start):
#     end = timedelta(hours=1)
#     end = start + end
#     delta = timedelta(minutes=1)
#     current = start
#     while current < end:
#         yield current
#         current += delta

datethen = "2020-11-02 11:49:21"
# datethen = datetime.strptime(datethen, '%Y-%m-%d %H:%M:%S')
# # print(bufdate)
# # print(dates)
#
# thenSlice = nowSlice.loc[nowSlice.belief_horizon_in_sec == datethen]
# print("Here",thenSlice)
#
# if thenSlice.empty:
#     bufdate = timedelta(minutes=1)
#     bufdate = datethen + bufdate
#     # print(bufdate)
#     dates = pd.date_range(datethen, bufdate, freq = "S")
#     # print(dates)
#     thenSlice = nowSlice.loc[nowSlice.belief_horizon_in_sec.isin(dates)]
#     print(thenSlice)


In [228]:
datenow = "2020-11-02 10:00:00"
datethen = "2020-11-02 11:49:21"
# now = "2020-11-02 10:00:00"
# if now:
#     print(now)

In [255]:
def getForcasts(now, then, df):
    """
    eturn the three kinds of forecasts that are most relevant for "then", given the knowledge you can assume was available at "now"
    :param now: type datetime, only thing we can assume to have format '%Y/%m/%d %H:%M:%S', "2020/11/02 10:33:33"
    :param then: type datetime, only thing we want
    :param df: type dataframe pandas,
    :return: temp, irri, wind
    """
    # datenow = now
    now = datetime.strptime(now, '%Y-%m-%d %H:%M:%S')
    nowSlice = df.loc[(df.event_start <= now)]
    # nowSlice
    # datethen = then
    then = datetime.strptime(then, '%Y-%m-%d %H:%M:%S')
    thenSlice = nowSlice.loc[nowSlice.belief_horizon_in_sec == then]
    if thenSlice.empty:
        bufdate = timedelta(seconds=30)
        end = now + bufdate
        start = now - bufdate
        dates = pd.date_range(start, end, freq="S")
        thenSlice = nowSlice.loc[nowSlice.belief_horizon_in_sec.isin(dates)]
    # irradiance = thenSlice.loc[thenSlice.sensor == "irradiance"]
    # wind = thenSlice.loc[thenSlice.sensor == "wind speed"]
    # tem = thenSlice.loc[thenSlice.sensor == "temperature"]
    # output = [i for i in zip(thenSlice.sensor, thenSlice.event_value)]
    output =  [i[0] + ": " + str(i[1]) for i in zip(thenSlice.sensor, thenSlice.event_value)]
    return output

In [256]:
print(getForcasts(datenow, datethen, df))

['temperature: 17.05', 'irradiance: 45.240923374577', 'wind speed: 10.63']


Given one parameter "now", a datetime, return three booleans, telling us if the next day (the one after "now") is expected to be "warm", "sunny" and "windy". Again, use only the knowledge from the dataset which you can assume was available at "now". Use three internal thresholds to determine the answer and you can also decide if the threshold (likely) being breached once is already enough for the boolean to be true.



In [231]:
datenow = "2021-07-01 20:00:00"
datenow = datetime.strptime(datenow, '%Y-%m-%d %H:%M:%S')
nowSlice = df.loc[(df.event_start <= datenow)]
nowSlice

,event_start,belief_horizon_in_sec,event_value,sensor,unit
0,2020-11-01 00:00:00,2020-10-31 23:49:23,11.36,temperature,°C
1,2020-11-01 00:00:00,2020-11-01 09:49:22,11.06,temperature,°C
2,2020-11-01 00:00:00,2020-11-01 10:49:20,11.41,temperature,°C
3,2020-11-01 00:00:00,2020-11-01 07:49:22,11.25,temperature,°C
4,2020-11-01 00:00:00,2020-11-01 00:49:29,11.39,temperature,°C
...,...,...,...,...,...
915236,2021-07-01 20:00:00,2021-07-01 23:55:00,2.99,wind speed,m/s
915237,2021-07-01 20:00:00,2021-07-01 22:55:00,2.99,wind speed,m/s
915238,2021-07-01 20:00:00,2021-07-01 21:55:00,2.99,wind speed,m/s
915239,2021-07-01 20:00:00,2021-07-01 20:56:00,2.88,wind speed,m/s


In [232]:
## get tomorrow
oneDay = timedelta(days=1)
tomor = datenow + oneDay
# aftertomor = tomor + oneDay
print(tomor.date())
# tomorSlice = nowSlice.loc[(nowSlice.belief_horizon_in_sec >= tomor.date()) and (nowSlice.belief_horizon_in_sec >= aftertomor.date())]
tomorSlice = nowSlice.loc[nowSlice.belief_horizon_in_sec.dt.date == tomor.date()]
print("Here", tomorSlice)



2021-07-02
Here                event_start belief_horizon_in_sec  event_value       sensor  \
95233  2021-06-30 02:00:00   2021-07-02 00:53:00        13.79  temperature   
95281  2021-06-30 03:00:00   2021-07-02 01:54:00        13.62  temperature   
95282  2021-06-30 03:00:00   2021-07-02 00:55:00        13.62  temperature   
95329  2021-06-30 04:00:00   2021-07-02 02:55:00        13.62  temperature   
95330  2021-06-30 04:00:00   2021-07-02 01:54:00        13.62  temperature   
...                    ...                   ...          ...          ...   
915231 2021-07-01 20:00:00   2021-07-02 04:55:00         2.84   wind speed   
915232 2021-07-01 20:00:00   2021-07-02 03:55:00         2.84   wind speed   
915233 2021-07-01 20:00:00   2021-07-02 02:55:00         2.99   wind speed   
915234 2021-07-01 20:00:00   2021-07-02 01:55:00         2.99   wind speed   
915235 2021-07-01 20:00:00   2021-07-02 00:55:00         2.99   wind speed   

       unit  
95233    °C  
95281    °C  
95282

In [233]:
dfIr = tomorSlice.loc[df["sensor"] == "irradiance"]
dfTem = tomorSlice.loc[df["sensor"] == "temperature"]
dfWind = tomorSlice.loc[df["sensor"] == "wind speed"]

If the mean of wind (sustained) speed is more than 11 m\s then it is windy

After light investigation more than 100 irradiance is considered sunny.

An average of more than 20 degrees C is considered warm

In [234]:
windy = True if dfWind.event_value.mean() >= 11 else False
windy

False

In [235]:
dfTem.event_value.mean()

15.58056878306879

In [236]:
def getTomorrow(now, df):
    """
    :param now: string in this format '%Y-%m-%d %H:%M:%S'
    :param df: dateset given
    :return: a list of three values or printing no information for tomorrow try again warm, sunny, windy
    """
    now = datetime.strptime(now, '%Y-%m-%d %H:%M:%S')
    nowSlice = df.loc[(df.event_start <= now)]
    oneDay = timedelta(days=1)
    tomor = now + oneDay
    tomorSlice = nowSlice.loc[nowSlice.belief_horizon_in_sec.dt.date == tomor.date()]
    if tomorSlice.empty:
        print("For the date given we do not have information please try another date")
    dfIr = tomorSlice.loc[df["sensor"] == "irradiance"]
    dfTem = tomorSlice.loc[df["sensor"] == "temperature"]
    dfWind = tomorSlice.loc[df["sensor"] == "wind speed"]
    warm = True if dfTem.event_value.mean() >= 20 else False
    sunny = True if dfIr.event_value.max() > 100 else False
    windy = True if dfWind.event_value.mean() >= 11 else False
    return warm, sunny, windy


In [237]:
datenow = "2021-07-01 20:00:00"
warm, sunny, windy = getTomorrow(datenow, df)
print("warm:", warm, "\nsunny:", sunny, "\nwindy:", windy)

warm: False 
sunny: True 
windy: False
